# Reinforcement Learning 2021-22: Second Group Assignment

**Deadline**: Mon 14 March 2022, 23:59

|Nr|**Name**|**Student ID**|**Email**|
|--|--------|--------------|---------|
|1.|Enzo Keuning        | 12878502             | gje.keuning@gmail.com        |
|2.|Hsi Yun Chien        | 12534919             | hsiyunchien@gmail.com        |
|3.| Jamie Mo        | 12475440             |    younjoo737@gmail.com     |

**Declaration of Originality**

We whose names are given under 1., 2., and 3. above declare that:

1. These solutions are solely our own work.
2. We have not made (part of) these solutions available to any other student.
3. We shall not engage in any other activities that will dishonestly improve my results or dishonestly improve or hurt the results of others.

## Instructions for completing and submitting the assignment
Please pay attention to the following instructions:

1. Please follow carefully the steps outlined in the assignment. If you cannot solve an exercise and this hinders continuing with subsequent exercises, try to find a way to work around it and give a clear explanation for the solution you have chosen.
2. Submit your work in the form of a Jupyter notebook and a html file (see point 6 below) via Canvas, before the deadline. Your notebook should not give errors when executed with `Run All`.
3. Most of your answers will consist of code. Make sure your code is well structured, efficient and provided with comments where needed. These aspects will be taken into account in the grading of your work.
4. Sometimes you are asked to answer some (open) questions. Please type your answer in the given Markdown cells and use your own words in answering those questions.
5. You are allowed to work on the assignment in groups of no more than 3 students and to submit together.
6. Save the complete work with all outputs in an HTML file: "File"->"Download as" -> "HTML (.html)" in Jupyter Notebook. Upload this HTML file to canvas as a supplement. You must submit BOTH notebook and HTML files.


## Taxi Problem

In this assignment, you will solve the taxi problem for the following map:

````
        +---------+
        |R: | : :G|
        | : | : : |
        | : : : : |
        | | : | : |
        |Y| : |B: |
        +---------+
````

There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends. The rewards are:

- -1 per step unless other reward is triggered.
- +20 delivering passenger.
- -10 executing "pickup" and "drop-off" actions illegally.

If a navigation action would cause the taxi to hit a wall (solid borders in the map), the action is a no-op, and there is only the usual reward of −1.

In [30]:
import numpy as np
from collections import defaultdict
from tqdm import tqdm
tqdm_disabled_episode=False

GroupNumber=33 # Input your group number Here

Let us import the ``TaxiEnv`` class from env_taxi.py and create an instance of the Taxi environment.

In [31]:
from env_taxi import TaxiEnv
env=TaxiEnv()

Then, we reset the environment:

In [32]:
np.random.seed(100)
env.reset()

(2, 3, 2, 1)

It returns four state variables:

- Row index of the taxi (starting from 0 in Python)
- Colum index of the taxi (starting from 0 in Python)
- Passenger location (0-4): 0=R, 1=G, 2=Y, 3=B, 4=in taxi
- Destination location (0-3): same encoding rule as that for passenger location but excluding the "in taxi" option.

You can use the ``describe_state`` method of the Taxi instance to display the state variables without location encoding.

In [33]:
env.describe_state()

{'Taxi Location': [2, 3], 'Passenger Location': 'Y', 'Destination Index': 'G'}

You can use the ``render`` method to visualize the state.

In [34]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



There are 6 discrete deterministic actions:
- 'South': move south
- 'North': move north
- 'East': move east
- 'West': move west
- 'Pickup': pickup passenger
- 'Dropoff': drop off passenger

In [35]:
print(env.action_space)

['South', 'North', 'East', 'West', 'Pickup', 'Dropoff']


Let us move one step to west:

In [36]:
env.step('West')

((2, 2, 2, 1), -1, False)

The output is a 3-tuple: the new state (a list of 4 numbers), reward, and whether the episode is ended. 

Let us visualize the new state. Note that the last action is shown at the bottom as well.

In [37]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


**Exercise 1** Complete the codes below to generate _one_ episode for the taxi driver who:
- Picks up the passenger when the taxi is at the location of the passenger when they are not yet at the destination;
- Drops off the passenger in the taxi when reaching the destination;
- Moves randomly with equal probabilities when finding the passenger or when the passenger is in the taxi (but not yet at destination). 

Then print the sum of rewards for the taxi driver in your episode.

In [38]:
state=env.reset(271) # The number 271 fixes the initial state. Do not change it.
env.render()
print(env.describe_state())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

{'Taxi Location': [2, 3], 'Passenger Location': 'Y', 'Destination Index': 'B'}


Let us program the policy function for the taxi driver:

In [39]:
def naive_policy(state):
    ## Start Coding Here
    dest = env.locs[state[3]]
    taxi = state[:2]
    if state[2] == 4: #passenger is in taxi
        if dest == taxi: #if the taxi is at the destination then 'dropoff'
            action = env.action_space[5]
        else: 
            action = np.random.choice(env.action_space[0:4]) #otherwise take a random action except 'dropoff'
    else:
        if env.locs[state[2]] == dest: #if the passenger is at the destination, then the taxi takes a random action except it cannot 'pickup' or 'dropoff'
            action = np.random.choice(env.action_space[0:4]) 
        elif taxi == env.locs[state[2]]: #if the taxi is at the passenger location, then 'pickup'
            action = env.action_space[4]
        else: 
            action = np.random.choice(env.action_space[0:4]) #else the taxi takes a random action
    ## End Coding Here                                      
    return action

In [40]:
# Import the simulate_episode function from env_taxi.py to help you simulate episode(s)
from env_taxi import simulate_episode
help(simulate_episode)

Help on function simulate_episode in module env_taxi:

simulate_episode(env, policy, max_iter=inf)
    Simulate a episode following a given policy
    @param env: game environment
    @param policy: policy (a dictionary or a function)
    @return: resulting states, actions and rewards for the entire episode



In [41]:
# Set the seed to make results reproducible
np.random.seed(GroupNumber)

states, actions, rewards = simulate_episode(env,naive_policy)

# Save the sum of rewards in this variable G
G=np.sum(rewards)


# Print the return for this episode
print("Return for this episode:", G)

Return for this episode: -10


**Exercise 2** Following the steps below to investigate whether we can solve the optimal policy by using Monte Carlo Exploring Starts. If possible, complete the codes and save the optimal (greedy) policy as a dictionary/defaultdict object. Otherwise, comment on the issues you encounter. 

Follow the instructions below to complete the exercise.

In [42]:
# We import the simulate_episode_ES function from env_taxi.py to simulate episodes with exploring starts
from env_taxi import simulate_episode_ES
help(simulate_episode_ES)


# Let us try to simulate one episode with exploring start for the naive policy from Exercise 1
# Print the states, actions and rewards for an episode (the first 200 steps only)
np.random.seed(GroupNumber)
states, actions, rewards=simulate_episode_ES(env, naive_policy, 200)
print(states, actions, rewards)

Help on function simulate_episode_ES in module env_taxi:

simulate_episode_ES(env, policy, max_iter=inf)
    Simulate a episode with exploring starts
    @param env: game environment
    @param policy: policy (a dictionary or a function)
    @param max_iter: maximum number of time steps
    @return: resulting states, actions and rewards for the entire episode

[(1, 1, 0, 3), (2, 1, 0, 3), (1, 1, 0, 3), (1, 1, 0, 3), (1, 1, 0, 3), (0, 1, 0, 3), (0, 1, 0, 3), (0, 1, 0, 3), (0, 0, 0, 3), (0, 0, 4, 3), (0, 1, 4, 3), (1, 1, 4, 3), (1, 0, 4, 3), (1, 0, 4, 3), (1, 1, 4, 3), (1, 1, 4, 3), (1, 0, 4, 3), (1, 1, 4, 3), (2, 1, 4, 3), (3, 1, 4, 3), (3, 2, 4, 3), (3, 2, 4, 3), (4, 2, 4, 3), (3, 2, 4, 3), (2, 2, 4, 3), (2, 1, 4, 3), (2, 2, 4, 3), (2, 3, 4, 3), (3, 3, 4, 3), (3, 3, 4, 3), (4, 3, 4, 3), (4, 3, 3, 3)] ['South', 'North', 'East', 'East', 'North', 'North', 'East', 'West', 'Pickup', 'East', 'South', 'West', 'West', 'East', 'East', 'West', 'East', 'South', 'South', 'East', 'East', 'South', '

If you can solve the optimal policy by using Monte Carlo Exploring Starts, write down your codes in the next cell and save the optimal (greedy) policy in the dictionary object ``pi_ES`` initialized below. Use no more than 50000 episodes. We will import the ``performance_evaluation`` function from env_taxi.py to evaluate your policy in new episodes (that terminate within 1000 time steps).

In [43]:
# Save your greedy actions as the values of this dictionary, whose keys are the states

#pi= defaultdict(lambda: np.random.choice(env.action_space))
#np.random.seed(GroupNumber)

# ## Start Coding Here 

#def mc_control_on_policy(env,gamma=1, n_episode=500): 
#    n_action = len(env.action_space)
#    G_sum = defaultdict(float)
#    N = defaultdict(int)
    
    # Initialization
    
#    Q = defaultdict(lambda: dict(zip(['South', 'North', 'East', 'West', 'Pickup', 'Dropoff'], np.empty(6))))
    
#    for episode in tqdm(range(n_episode)):
#        states_t, actions_t, rewards_t = simulate_episode(env)
        
        # Calculate the returns for all state-action pairs in this episode
        
#        return_t = 0
#        G = {}
#        for state_t, action_t, reward_t in zip(states_t[::-1][1:],rewards_t[::-1]):# remove the last one 
#            return_t = gamma * return_t + reward_t
#            G[(state_t, action_t)] = return_t

        # Update the action-value estimate and policy
    
#        for state_action, return_t in G.items():
#            state, action = state_action
#            if state[2] != state[3]:
#                G_sum[state_action] += return_t
#                N[state_action] += 1
#                Q[state][action] = G_sum[state_action] / N[state_action]
#                pi[state]=max(Q[state],key=Q[state].get)
#    return Q, pi

## End Coding Here
 


# # We import the performance_evaluation function from env_taxi.py to evaluate your optimal policy
#from env_taxi import performance_evaluation
#performance_evaluation(env,pi)

If you cannot solve the optimal policy by using Monte Carlo Exploring Starts, explain the issue(s) and why they cannot be addressed.

Answer: Monte Carlo Exploring Starts is looping forever for each episode, choosing states randomly such that all the pairs of state and action have a probability >0, however in this question, there are some actions that are not allowed to go from certain states. Like for example, the driver couldnt "Pickup" if he hasn't arrived in location of passenger. Therefore, we could not solve the optimal policy using monte carlo exploring starts. 
(We still give the potential code, but inserted hashtags so it is not evaluated).

**Exercise 3**:  Find the optimal policy by using Q-learning with 50000 episodes and the exploration probability $\varepsilon=0.1$. Try two different values of the step-size parameter $\alpha=0.4$ and $\alpha=0.1$. Compare their performance in 10000 new episodes and comment on the similarities or/and differences. 

Follow the instructions below to complete the exercise.

In [44]:
np.random.seed(GroupNumber)

## Start Coding Here: Find the optimal policy by using Q-learning with 50000 episodes
def gen_epsilon_greedy_policy(action_space, Q, epsilon):
    """
    Generate epsilon greedy policy as a function
    @param action_space: list of all actions
    @param Q: the action values, Q[state] is a dictitonary
    @param epsilon: value of epsilon
    @return: epsilon-greedy policy as a function
    """
    def epsilon_greedy_policy(state):
        greedy_action=max(Q[state],key=Q[state].get)
        random_action=np.random.choice(action_space)
        return np.random.choice([random_action,greedy_action],p=[epsilon,1-epsilon])
    
    return epsilon_greedy_policy

def q_learning(env, gamma, n_episode, alpha, epsilon=0.1):
    """
    Off-policy Q-learning
    @param env: game environment
    @param gamma: discount factor
    @param n_episode: number of episodes
    @param alpha: step-size parameter
    @param epsilon: exploring probability
    @return: the optimal Q-function, the optimal policy, length of episode and the sum of rewards per episode
    """
    length_episode = np.zeros(n_episode) 
    sum_reward_episode = np.zeros(n_episode)
    
    Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space)))))
    epsilon_greedy_policy = gen_epsilon_greedy_policy(env.action_space, Q, epsilon)
    
    pi= defaultdict(lambda: np.random.choice(env.action_space))
    
    for episode in tqdm(range(n_episode), disable=tqdm_disabled_episode):
        ## Start Coding Here
        
        state = env.reset()
        is_done = False
        # Generate behavior using epsilon greedy policy
        action = epsilon_greedy_policy(state)
        while not is_done:
            next_state, reward, is_done = env.step(action)
            next_action = epsilon_greedy_policy(next_state)
            
            # A special case of off-policy Expected Sarsa with epsilon=0
            v= max(Q[next_state].values())
            
            td_delta = reward + gamma * v - Q[state][action]
            
            Q[state][action] += alpha * td_delta
            # Updte Target Policy
            pi[state]=max(Q[state],key=Q[state].get)
            
            length_episode[episode] += 1
            sum_reward_episode[episode] += reward
            if is_done:
                break
            state = next_state
            action = next_action
            
        ## End Coding Here    
    return Q, pi, length_episode, sum_reward_episode

# Save the policy for alpha=0.4 as a Python dictionary/defaultdic object pi_qlearning_1
Q_qlearning_1, pi_qlearning_1, length_episode_qlearning_1, sum_reward_episode_qlearning_1 = q_learning(env, 1, 50000, 0.4, 0.1)

# Save the policy for alpha=0.1 as a Python dictionary/defaultdic object pi_qlearning_2
Q_qlearning_2, pi_qlearning_2, length_episode_qlearning_2, sum_reward_episode_qlearning_2 = q_learning(env, 1, 50000, 0.1, 0.1)


## End Coding Here

100%|██████████| 50000/50000 [01:06<00:00, 756.76it/s]


In [45]:
# Compare their performance in 10000 new episodes: Do you find a big difference?
from env_taxi import performance_evaluation

print('--------  Q-learning, alpha=0.4  --------')
performance_evaluation(env,pi_qlearning_1)

print('--------  Q-learning, alpha=0.1  --------')
performance_evaluation(env,pi_qlearning_2)

  4%|▍         | 399/10000 [00:00<00:04, 2017.55it/s]

--------  Q-learning, alpha=0.4  --------


  4%|▍         | 419/10000 [00:00<00:04, 2107.23it/s]

The sum of rewards per episode: 7.8718
The percentage of episodes that cannot terminate within 1000 steps: 0.0
--------  Q-learning, alpha=0.1  --------


100%|██████████| 10000/10000 [00:04<00:00, 2101.00it/s]

The sum of rewards per episode: 7.8886
The percentage of episodes that cannot terminate within 1000 steps: 0.0


Commment on the similarities or/and differences.

Answer: We see that both the Q-learning method with a learning rate of $\alpha=0.1$ and $\alpha=0.4$ always terminate within 1000 steps, so the taxi always managed to dropoff the passenger at destination within 1000 iterations. However the sum of the rewards per episode are slightly higher in the method with the smaller learning rate ($\alpha=0.1$), this implies that on average it took slightly less time/iterations, or/and with less mistakes, for the taxi to bring the passenger to the right destination with the Q-learning method that has a learning rate of $\alpha=0.1$ compared to the method with a learning rate of $\alpha=0.4$. 

**Exercise 4**: Follow the steps below to find the optimal $\varepsilon$-soft policy by using Sarsa with 50000 episodes and the exploration probability $\varepsilon=0.1$. Try two different values of the step-size parameter $\alpha=0.4$ and $\alpha=0.1$. Compare their performance, and comment on the similarties or/and differences with that of Q-learning.

Follow the instructions below to complete the exercise.

In [46]:
np.random.seed(GroupNumber)

## Start Coding Here
def gen_epsilon_greedy_policy(action_space, Q, epsilon):
    def epsilon_greedy_policy(state):
        greedy_action=max(Q[state],key=Q[state].get)
        random_action=np.random.choice(action_space)
        return np.random.choice([random_action,greedy_action],p=[epsilon,1-epsilon])
    
    return epsilon_greedy_policy


# Save the policy for alpha=0.4 in a Python function pi_sarsa_1
# Save the policy for alpha=0.1 in a Python function pi_sarsa_2


def sarsa(env, gamma, n_episode, alpha, epsilon):
    """
    Obtain the optimal policy with on-policy SARSA algorithm
    @param env: game environment
    @param gamma: discount factor
    @param n_episode: number of episodes
    @param alpha: step-size parameter
    @param epsilon: exploring probability
    @@return: the optimal Q-function, the optimal epsilon-greedy policy (as a function), length of episode and the sum of rewards per episode
    """
    
    Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space)))))
    
    pi = gen_epsilon_greedy_policy(env.action_space, Q, epsilon)
    
    for _ in tqdm(range(n_episode), disable=tqdm_disabled_episode):
        
        state = env.reset()
        is_done = False
        action = pi(state)
        while not is_done: 
            #getting the next state
            next_state, reward, is_done = env.step(action)
            
            #choosing the next action using the target policy (epsilon-greedy here)
            next_action = pi(next_state)
            
            td_delta = reward + gamma * Q[next_state][next_action] - Q[state][action]
            #Learning the Q-value, decomposed formula
            Q[state][action] += alpha * td_delta
            
            if is_done:
                break
            state = next_state
            action = next_action
            
    return Q, pi

# End Coding Here
Q_sarsa_1, pi_sarsa_1 = sarsa(env, 1, 50000, 0.4, 0.1)
Q_sarsa_2, pi_sarsa_2 = sarsa(env, 1, 50000, 0.1, 0.1)


100%|██████████| 50000/50000 [01:02<00:00, 796.13it/s]


In [47]:
from env_taxi import performance_evaluation

print('--------  Sarsa, alpha=0.4  --------')
performance_evaluation(env,pi_sarsa_1)

print('--------  Sarsa, alpha=0.1  --------')
performance_evaluation(env,pi_sarsa_2)

  1%|          | 62/10000 [00:00<00:38, 255.16it/s]

--------  Sarsa, alpha=0.4  --------


  2%|▏         | 165/10000 [00:00<00:11, 843.30it/s]

The sum of rewards per episode: -37.44326383319968
The percentage of episodes that cannot terminate within 1000 steps: 0.0024
--------  Sarsa, alpha=0.1  --------


100%|██████████| 10000/10000 [00:12<00:00, 811.79it/s]

The sum of rewards per episode: -0.883
The percentage of episodes that cannot terminate within 1000 steps: 0.0


Compare their performance, and comment on the similarties or/and differences with that of Q-learning.

Answer: We observe that SARSA with the larger learning rate, $\alpha = 0.4$, a small fraction of the episodes does not terminate, which implies that in some episodes that taxi is not able to dropoff the passenger within 1000 steps. This is not the case when a smaller learning rate is chosen, $\alpha = 0.1$, then all episodes terminate within a 1000 steps. From theory, we know that a smaller learning rate will take more time to converge, and a higher learning rate adjusts more abruptly, which might lead to non-convergence of the algorithm. 
Decreasing the learning rate, $\alpha$, from 0.4 to 0.1 has a tremendous effect on the results of the SARSA algorithm. When a learning rate of 0.4 is used the sum of the rewards per episode it approximately -37.44, whereas with a smaller learning rate of 0.1 the sum of the rewards per episode is a negative number with a dramatically smaller absolute value. Which implies that a smaller learning rate works remarkably better in the SARSA algorithm. Intuitively, it makes sense that alpha influences SARSA to a greater extent as SARSA is an on-policy algorithm and Q-learning an off-policy algorithm.

In contrast, in Q-learning we observed a positive sum of rewards per episode for both values for $\alpha$, with only a small-scale positive change when taking the smaller stepsize, $\alpha = 0.1$. Q-learning is an off-policy algorithm that is more agressive, in the sense that it is willing to take more risk to have a quick iterating algorithm. Whereas the SARSA algorithm will take less risk, resulting in a longer, but safer iteration. 

In the taxi-problem mistakes are relatively not that costsly (for example in comparison with the Cliff Waling problem form the book), hence it makes intuitive sense that Q-learning gives a higher sum of rewards per episode. 


**Exercise 5**: Do you expect double Q-learning will improve Q-learning substantially? Motivate your answer. You do not need to implement the double Q-learning algorithm.

Answer: 

Yes. All the algorithms we implement for control problems involve maximizing the target policy. For example, the target policy of Q learning is greedy policy ,whereas the target policy of SARSA is epsilon-greedy policy. The fact that these algorithms try to max over all the actions ( and use the maximum value of the estimated action-value as the maximum value of the actual action-value) overestimate the value function estimates and action-value estimates, which is known as the maximization bias problem. 

To avoid causing such large bias, double Q-learning divides the sample, one for generating independent estimates and their actual values, the other one for determining the optimal action. This approach is then unbiased. 

However, in the taxi-problem we have that all rewards are non-stochastic, hence we do not expect a substantial overestimation of the value function estimates and action-value estimates. Therefore, we expect that double Q-learning will not improve Q-learning substantially. 




**Exercise 6** 
Follow the steps below to investigate whether we can solve the optimal policy by using off-policy Monte Carlo control with weighted importance sampling. Use the optimal $\varepsilon$-soft policy found by Sarsa above with $\alpha=0.1$ as the behavior policy and use no more than 50000 episodes.

In [48]:
np.random.seed(GroupNumber)

## Start Coding Here

# Save your optimal policy as a dictionary/defaultdict Python object pi_wis
def mc_control_off_policy_weighted(env, n_episode, behaviour_policy, Q_sarsa, gamma = 1):

    C = defaultdict(float) # initial value=0
    
    # Initialization
    Q = defaultdict(lambda: dict(zip(env.action_space, np.empty(len(env.action_space)))))
    pi= defaultdict(lambda: np.random.choice(env.action_space))
    
    for _ in tqdm(range(n_episode)):
        states_t, actions_t, rewards_t = simulate_episode(env, behaviour_policy)
        
        W = 1.
        return_t = 0.
        for state_t, action_t, reward_t in zip(states_t[::-1][1:], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            C[(state_t, action_t)] += W
            Q[state_t][action_t] += (W / C[(state_t, action_t)]) * (return_t - Q[state_t][action_t])
            pi[state_t]=max(Q[state_t],key=Q[state_t].get)
            
            if action_t != pi[state_t]:
                break
            elif action_t == max(Q_sarsa[state_t], key=Q_sarsa[state_t].get):
                b = (1-0.1)+0.1*(1/len(env.action_space))
            else:
                b = 0.1*(1/len(env.action_space))
            W *= 1./ b
                
    return Q, pi





## End Coding Here

Check the total number of distinct states visited in your episodes. Have you reached all 400 possible states (within the episode)?

In [49]:
Q_wis, pi_wis = mc_control_off_policy_weighted(env, 50000, pi_sarsa_2, Q_sarsa_2)

print('Number of States Visited:', len(pi_wis))

100%|██████████| 50000/50000 [01:03<00:00, 788.65it/s]

Number of States Visited: 347


Generate a complete policy that:
- follows your optimal policy (in this exercise) if the state is visited in your episodes (in this exercise);
- otherwise follows the naive policy in Exercise 1.

If you have reached all possible states in your episodes, it is identical to your optimal policy.

In [50]:
def complete_policy(state):
    ## Start Coding Here
    if state in Q_wis:
        action = pi_wis[state]
    else:
        action = naive_policy(state)    
    ## End Coding Here
    return action

Let us evaluate its performance:

In [51]:
performance_evaluation(env,complete_policy)

100%|██████████| 10000/10000 [00:29<00:00, 343.68it/s]

The sum of rewards per episode: 7.636850152905199
The percentage of episodes that cannot terminate within 1000 steps: 0.0844


**Exercise 7**: What are your conclusions from the analysis above? What are the advantages and disadvantages of these methods?

Answer: We can see that the optimal policy by using off-policy Monte Carlo control with weighted importance sampling
with a stepsize of $\alpha=0.1$ only reached 347 states. The sum of the rewards per episodes in the complete policy, including off-policy MC and the initial naive policy, was only marginally smaller than of Q-learning. This could be interpreted as casusing slightly more time for the taxi to bring the passenger to the right destination, or making a little more mistakes along the way compared to Q-learning. But, also note that the complete ploicy from q6 has a considerable higher percentage of episodes that cannot terminate within a 1000 iterations.  

The sum of rewards per episode was substantially smaller and negative in the optimal $\varepsilon$-soft policy found by Sarsa. 
Hence, it seems like the Q-learning method gives the most optimal policy in the sense that it gives the highest rewards per episode and it always terminates within 1000 steps. 

As earlier mentioned, the Q-learning method is riskier as it has a higher variance, but it may also converge quicker. In contrast, the Sarsa method will take less risk and may converge slower, but safer. The MC off-policy method does not visit all states and it can only update when the return at the end of an episode is known. 
In the taxi-problem mistakes are relatively incostly, hence the fast-iterating Q-learning method gives the best results. 